In [3]:
# !pip install -qU python-gdcm pydicom pylibjpeg
# !pip install -U pylibjpeg-libjpeg -v
# !pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg
# !pip install pydicom

# import include nesscessaryz
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

## RSNA

In [9]:
meta_rsna = pd.read_csv(r"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\rsna.csv")

In [12]:
meta_rsna= meta_rsna[['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']]
meta_rsna[meta_rsna.patient_id == 5]

,patient_id,image_id,view,laterality,age,cancer
37282,5,640805896,MLO,L,68.0,0
37283,5,1351088028,CC,L,68.0,0
37284,5,940388076,CC,R,68.0,0
37285,5,1633417959,MLO,R,68.0,0


In [11]:
for i in range(len(meta_rsna)):
    print(meta_rsna.image_id[i])

462822612
1459541791
1864590858
1874946579
220375232
270344397
541722628
1031443799
562340703
1365269360
288394860
893612858
1967300488
2142944869
850559196
1350492010
102733848
202939811
1648588715
294481519
495770405
964141995
1234933874
1577142909
1842203124
349510516
906829939
1464813991
94335194
1207499426
1351270472
588678397
1749389520
1428987847
1614607569
938549206
1238319934
833024366
2046475482
51088550
1973073874
1479874325
2029358943
1450760951
1854592291
25264463
1276618392
664674273
929754876
1957744616
166011602
268368896
1007799858
618254763
1245250349
1304802631
1480506800
453020471
1181635673
1241778584
1951607544
238675415
2057295788
190315990
929085221
472095321
539757861
527043166
1470873094
50833170
494229141
394215927
1854075671
722196386
1732550501
612913905
828772252
1891054212
2029770528
473162373
1919498169
23590109
1495487293
1987272730
677689694
808365199
1035012441
388811999
613462606
1360338805
1672636630
1013166704
1165309236
2110820077
726735438
164638

In [69]:
# Tổng quan bệnh nhân 
quantity_par = meta_rsna.groupby(['patient_id', 'view', 'laterality'])['image_id'].count()
quantity_par.to_frame()

image_id
patient_id view laterality          
5          CC   L                  1
                R                  1
           MLO  L                  1
                R                  1
25         CC   L                  1
...                              ...
65530      MLO  R                  2
65534      CC   L                  1
                R                  1
           MLO  L                  1
                R                  1

[47688 rows x 1 columns]

In [70]:
len(meta_rsna.patient_id.unique())

11913

In [71]:
# Quantity to views CC vs MLO
quantity_image_par = meta_rsna.groupby(['patient_id'])['image_id'].count()
quantity_image_par.to_frame().value_counts()


image_id
4           8233
5           1767
6           1218
7            276
8            229
9             99
10            53
11            19
12            12
13             5
14             2
dtype: int64

In [72]:
## Chech CC vs ML0
meta_rsna.view.value_counts()

MLO    27903
CC     26765
AT        19
LM        10
ML         8
LMO        1
Name: view, dtype: int64

In [73]:
## Chech Left of CC vs MLO
meta_rsna.groupby(['view', 'laterality', 'cancer'])['image_id'].count().to_frame()

image_id
view laterality cancer          
AT   L          0              7
     R          0             10
                1              2
CC   L          0          13009
                1            286
     R          0          13190
                1            280
LM   L          0              4
     R          0              6
LMO  R          0              1
ML   L          0              3
     R          0              5
MLO  L          0          13656
                1            302
     R          0          13657
                1            288

### Note:
+ Per patient in rsna have image from 4 

In [74]:
quantity_image_par.to_frame()

,image_id
patient_id,
5,4
25,6
28,4
30,4
33,5
...,...
65518,4
65525,4
65526,4


## Analyst for cancer RSNA 

In [75]:
meta_cancer = meta_rsna[meta_rsna.cancer == 1].reset_index()
meta_cancer

,index,patient_id,image_id,view,laterality,age,cancer
0,87,10130,388811999,MLO,L,71.0,1
1,88,10130,613462606,CC,L,71.0,1
2,89,10130,1360338805,CC,L,71.0,1
3,90,10130,1672636630,MLO,L,71.0,1
4,190,10226,461614796,MLO,L,71.0,1
...,...,...,...,...,...,...,...
1153,54485,9707,1900919592,MLO,R,56.0,1
1154,54593,9840,111786935,MLO,R,59.0,1
1155,54594,9840,1559360810,CC,R,59.0,1
1156,54595,9851,1409270544,MLO,L,58.0,1


In [76]:
# Quantity to views CC vs MLO
quantity_image_cancer = meta_cancer.groupby(['patient_id'])['image_id'].count()
quantity_image_cancer = quantity_image_cancer.to_frame()
quantity_image_cancer.value_counts()

image_id
2           343
3           107
4            29
5             7
dtype: int64

## check with quantity img equal 2


In [77]:
alna2 = quantity_image_cancer[quantity_image_cancer.image_id == 2]
alna2

,image_id
patient_id,
106,2
236,2
283,2
500,2
729,2
...,...
65198,2
65224,2
65244,2


In [78]:
meta_alna2 = meta_cancer[meta_cancer.patient_id.isin(alna2.image_id.index)].reset_index()
meta_alna2

,level_0,index,patient_id,image_id,view,laterality,age,cancer
0,4,190,10226,461614796,MLO,L,71.0,1
1,5,191,10226,530620473,CC,L,71.0,1
2,6,215,1025,773597682,CC,L,56.0,1
3,7,216,1025,1803952236,MLO,L,56.0,1
4,8,407,10432,458553546,CC,L,65.0,1
...,...,...,...,...,...,...,...,...
681,1153,54485,9707,1900919592,MLO,R,56.0,1
682,1154,54593,9840,111786935,MLO,R,59.0,1
683,1155,54594,9840,1559360810,CC,R,59.0,1
684,1156,54595,9851,1409270544,MLO,L,58.0,1


In [79]:
## check CC vs MLO
meta_alna2.view.value_counts()

MLO    343
CC     343
Name: view, dtype: int64

Because, Quantity view is different CC and MLO is very small, so  I will remove it 

In [148]:
## Quantity image left of MLO == left of CC and right of MLO == right of CC
meta_par = meta_alna2.groupby(['patient_id', 'view', 'laterality']).size()
meta_par[(106, 'MLO', 'L')]

1

In [152]:
par_id = meta_alna2.patient_id.unique()
cnt_full = 0
cnt_mis = 0

for id in par_id:
    try:
        if meta_par[(id, 'MLO', 'L')] == meta_par[(id, 'CC', 'L')]:
            cnt_full+=1
    except:
        cnt_mis+=1
    try:
        if meta_par[(id, 'MLO', 'R')] == meta_par[(id, 'CC', 'R')]:
            cnt_full+=1
    except:
        cnt_mis+=1


In [153]:
cnt_full

343

In [154]:
cnt_mis

343

In [136]:
for i in meta_par:
    print(i)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [139]:

count_cc_mlo = {}
for i in range(len(meta_new)):
    id = tuple(meta_new.patient_id[i], meta_new.view[i], meta_new.laterality[i])
    # print(id)
    count_cc_mlo.update({id, meta_new.Quantity[i]})

TypeError: tuple expected at most 1 argument, got 3

In [118]:
meta_new

,patient_id,view,laterality,0
0,106,CC,L,1
1,106,MLO,L,1
2,236,CC,L,1
3,236,MLO,L,1
4,283,CC,R,1
...,...,...,...,...
681,65244,MLO,R,1
682,65427,CC,L,1
683,65427,MLO,L,1
684,65492,CC,L,1


In [102]:
id = meta_par.index[2]
id

(236, 'CC', 'L')